In [19]:
import os
from urllib.request import urlretrieve
from multiprocessing import Pool
from collections import defaultdict, namedtuple
from pprint import pprint
import tqdm
import vk
import json
from vk import api


In [10]:

def get_password_and_id(cp):
    creds = json.load(open(cp))
    return creds['pass'], creds['id']

password, id = get_password_and_id('creds.json')

session = vk.AuthSession(app_id='6787646', user_login=id,
                         scope='messages', user_password=password) 
vkapi = vk.API(session)

In [21]:
audioapi = vk.VkAudio(vkapi)
audioapi.get(owner_id=9468306)

AttributeError: module 'vk' has no attribute 'VkAudio'

In [61]:
path = 'dumps/nikkorobk/9468306/messages.json'

In [62]:
f = json.load(open(path))

# count says 6529

In [63]:
f['items'] = f['items'][500:]
f['count'] = f['count']-500

In [64]:
json.dump(f, open(path,'w'), ensure_ascii=False )

In [65]:
len(f['items']), f['count']


(6029, 6029)

In [66]:

f['items'][199]

{'date': 1543332686,
 'from_id': 9468306,
 'id': 281828,
 'out': 0,
 'peer_id': 9468306,
 'text': 'Мне не нравится, что она еле сказала мне правду вчера, скрывает, как от маленькой.',
 'conversation_message_id': 5832,
 'fwd_messages': [],
 'important': False,
 'random_id': 0,
 'attachments': [],
 'is_hidden': False}